In [1]:
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
data = image_dataset_from_directory("gdrive/MyDrive/waiste data", 
                             labels='inferred', label_mode='categorical', class_names=None, color_mode='rgb',
                             batch_size=32, image_size=(150, 150), shuffle=True, seed=None,
                             validation_split=None, subset=None, interpolation='bilinear', follow_links=False,
                             crop_to_aspect_ratio=False)

Found 22610 files belonging to 10 classes.


In [4]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

data_norm = data.map(process)

In [5]:
data_sample = data.take(300)

data_im = []
data_lb = []

for im, lb in data_sample:
  data_im.append(im)
  data_lb.append(lb)

In [6]:
def norma_2(image):
  image = tf.cast(image/255, tf.float32)
  return image

In [7]:
data_im_2 = [norma_2(image) for image in data_im]

In [8]:
data_im_2 = np.array(data_im_2)
data_lb = np.array(data_lb)

In [9]:
data_im_2 = data_im_2.reshape(-1, 150, 150, 3)
data_lb = data_lb.reshape(-1, 10)

In [10]:
# Split the data

from sklearn.model_selection import train_test_split

train_X, valid_X, train_y, valid_y = train_test_split(data_im_2, data_lb, test_size=0.3, random_state=42)
valid_X, test_X, valid_y, test_y = train_test_split(valid_X, valid_y, test_size=0.5, random_state=42)

## Build Model

In [11]:
from tensorflow.keras.applications import inception_resnet_v2
from matplotlib.pyplot import plt
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, MaxPool2D, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

ImportError: ignored

In [ ]:
inception_resnet_v2 = inception_resnet_v2.InceptionResNetV2(
    include_top=False, weights="imagenet", input_tensor=None,
    input_shape=(150, 150, 3), pooling=None, classifier_activation='softmax')

In [ ]:
inception_resnet_v2.trainable = False

In [ ]:
def init_model():
  model = Sequential()
  model.add(Input(shape=(150, 150, 3)))
  model.add(inception_resnet_v2)
  model.add(Conv2D(32, (4,4), padding='same', activation="relu"))
  model.add(MaxPool2D(pool_size=(3,3)))
  model.add(Dropout(0.4))
  model.add(Conv2D(32, (3,3), padding='same', activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.3))
  model.add(Conv2D(32, (3,3), padding='same', activation="relu"))
  model.add(Flatten())
  model.add(Dense(50, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
  return model

In [ ]:
model = init_model()

In [ ]:
model.summary()

In [ ]:
es = EarlyStopping(patience = 10, monitor='val_categorical_accuracy' , mode='max', min_delta=0.001, restore_best_weights = True)

In [ ]:
history = model.fit(train_X, train_y, validation_split = 0.2, callbacks = [es], batch_size = 32, epochs = 100, verbose = 1)

In [ ]:
def summarize_diagnostics(history):
  # plot loss
  plt.figure(figsize=(10, 15))
  plt.subplot(211)
  plt.title('Cross Entropy Loss')
  plt.plot(history.history['loss'], color='blue', label='train')
  plt.plot(history.history['val_loss'], color='orange', label='test')
  # plot accuracy
  plt.subplot(212)
  plt.title('Classification Accuracy')
  plt.plot(history.history['accuracy'], color='blue', label='train')
  plt.plot(history.history['val_accuracy'], color='orange', label='test')

In [ ]:
summarize_diagnostics(history)

In [ ]:
pickle.dump(model, open('InceptionResNet_V2.pickle', mode='wb'))